In [63]:
import pymongo
MongoClient = pymongo.MongoClient
client = MongoClient()
db = client.companies

#### 1. filtramos por categoría(web,software y games), empresas creadas en los últimos 10 años, que no hayan quebrado y que el número de empleados no sea nulo
#### 2. por cada sucursal creamos un documento nuevo utilizando "unwind"
#### 3. con "project" seleccionamos los datos que nos interesan para el df

In [92]:
cursor=db.companies.aggregate([
    {'$match': 
     {'$and': [ 
        {'category_code': {'$in': ["web","software","games_video"]}},
        {'founded_year':{'$gte':2009}},
        {'deadpooled_year': None },
        {"offices.latitude":{'$ne': None}},
        {"offices.longitude":{'$ne': None}},
        {"number_of_employees":{'$ne': None}},
         
    ]}},
    { '$unwind': "$offices" },
    { '$project' : {
        "name" : 1,
        "category_code": 1,
        "founded_year": 1,
        "description":1,
        "money_raised":1,
        "number_of_employees":1,
        "offices.latitude": 1,
        "offices.longitude":1,
        "offices.country_code":1,
        "_id":0
    }}])

Comp=list(cursor)

In [93]:
import pandas as pd

df = pd.DataFrame(Comp)
df

,category_code,description,founded_year,name,number_of_employees,offices
0,web,Tech Support Community,2013,Fixya,30,"{'country_code': 'USA', 'latitude': 37.566879,..."
1,games_video,,2009,alluc,7,"{'country_code': 'DEU', 'latitude': 53.707739,..."
2,games_video,social gaming platform,2011,Social Gaming Network,100,"{'country_code': 'USA', 'latitude': 37.446823,..."
3,games_video,social gaming platform,2011,Social Gaming Network,100,"{'country_code': 'USA', 'latitude': 34.0698494..."
4,games_video,Educational Games,2009,Crootpad,2,"{'country_code': 'USA', 'latitude': 42.091078,..."
5,software,Financial Engineering Platform,2009,Macroaxis,8,"{'country_code': 'USA', 'latitude': 37.7896292..."
6,games_video,People Powered Radio,2011,Fuzz,6,"{'country_code': 'USA', 'latitude': 37.760524,..."
7,web,Take Control of Your Own Search Results,2009,BrandYourself,22,"{'country_code': 'USA', 'latitude': 42.275263,..."
8,web,College Life Online,2010,CollegeConvo,2,"{'country_code': 'USA', 'latitude': 40.650291,..."
9,software,Big data automotive marketing platform,2012,Carfeine,5,"{'country_code': 'USA', 'latitude': 38.989124,..."


#### Preparamos columnas de longitud y latitud para Tableau

In [94]:
df['lat'] = df.apply(lambda row: row['offices']['latitude'], axis=1)
df['long'] = df.apply(lambda row: row['offices']['longitude'], axis=1)
df['country'] = df.apply(lambda row: row['offices']['country_code'], axis=1)
df['coord'] = df[['lat', 'long']].apply(list, axis=1)
df=df.drop(columns=["offices"])
df.head()


,category_code,description,founded_year,name,number_of_employees,lat,long,country,coord
0,web,Tech Support Community,2013,Fixya,30,37.566879,-122.323895,USA,"[37.566879, -122.323895]"
1,games_video,,2009,alluc,7,53.707739,10.023246,DEU,"[53.707739, 10.023246]"
2,games_video,social gaming platform,2011,Social Gaming Network,100,37.446823,-122.161523,USA,"[37.446823, -122.161523]"
3,games_video,social gaming platform,2011,Social Gaming Network,100,34.069849,-118.405418,USA,"[34.0698494, -118.4054182]"
4,games_video,Educational Games,2009,Crootpad,2,42.091078,-88.288749,USA,"[42.091078, -88.288749]"


In [95]:
def toGeoJSON(array):
    return {
        "type":"Point",
        "coordinates":[array[1],array[0]]}
df['position'] = df.apply(lambda r:toGeoJSON(r['coord']),axis=1)
df=df.drop(columns=["coord"])

In [96]:
df.head()

,category_code,description,founded_year,name,number_of_employees,lat,long,country,position
0,web,Tech Support Community,2013,Fixya,30,37.566879,-122.323895,USA,"{'type': 'Point', 'coordinates': [-122.323895,..."
1,games_video,,2009,alluc,7,53.707739,10.023246,DEU,"{'type': 'Point', 'coordinates': [10.023246, 5..."
2,games_video,social gaming platform,2011,Social Gaming Network,100,37.446823,-122.161523,USA,"{'type': 'Point', 'coordinates': [-122.161523,..."
3,games_video,social gaming platform,2011,Social Gaming Network,100,34.069849,-118.405418,USA,"{'type': 'Point', 'coordinates': [-118.4054182..."
4,games_video,Educational Games,2009,Crootpad,2,42.091078,-88.288749,USA,"{'type': 'Point', 'coordinates': [-88.288749, ..."


#### Exportamos df a fichero json

In [97]:
df.to_json('companies_filtered.json', orient="records", lines=True)

#### Sacamos listado con empresas que tienen más compañías a su alrededor. 

In [98]:
cursor=db.comp.find()
distance=[]
for i in cursor:
    cursor2 = db.comp.find({
  "position": {
    "$near": {
      "$geometry": {
        "type": "Point",
        "coordinates": [i['position']['coordinates'][0],i['position']['coordinates'][1]]},
      "$minDistance": 0,
      "$maxDistance":20000
    }}})
    distance.append(cursor2.count())
print(distance)
    



/usr/local/Cellar/ipython/7.2.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:13: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  del sys.path[0]


[5, 4, 7, 5, 1, 7, 7, 3, 3, 3, 7, 1, 7, 3, 1, 7, 1, 1, 9, 1, 7, 9, 1, 2, 7, 1, 9, 1, 3, 2, 1, 2, 3, 1, 1, 1, 9, 1, 2, 1, 2, 7, 6, 2, 1, 1, 1, 5, 3, 1, 5, 1, 1, 5, 9, 2, 4, 5, 7, 2, 1, 1, 5, 5, 8, 5, 2, 2, 2, 2, 2, 2, 1, 1, 2, 5, 1, 1, 3, 1, 1, 1, 2, 3, 1, 4, 2, 1, 1, 1, 1, 2, 9, 1, 1, 4, 1, 1, 9, 3, 5, 5, 5, 2, 7, 1, 1, 3, 9, 5, 1, 2, 5, 1, 1, 1, 3, 1, 3, 9, 1, 3, 5, 1]


#### La lista "distance" contiene aquellas empresas que tienen más de 6 empresas a su alrededor en 20.000km

In [99]:
cursor=db.comp.find()
distance=[]
for i in cursor:
    cursor2 = db.comp.find({
  "position": {
    "$near": {
      "$geometry": {
        "type": "Point",
        "coordinates": [i['position']['coordinates'][0],i['position']['coordinates'][1]]},
      "$minDistance": 0,
      "$maxDistance":20000
    }}})
    if cursor2.count()>6:
        distance.append(i)
import pandas as pd
df=pd.DataFrame(list(distance))
df

/usr/local/Cellar/ipython/7.2.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:13: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  del sys.path[0]


,_id,category_code,country,description,founded_year,lat,long,name,number_of_employees,position
0,5c6af0068f92e5078765f1c1,games_video,USA,social gaming platform,2011,37.446823,-122.161523,Social Gaming Network,100,"{'type': 'Point', 'coordinates': [-122.161523,..."
1,5c6af0068f92e5078765f1c4,software,USA,Financial Engineering Platform,2009,37.789629,-122.399878,Macroaxis,8,"{'type': 'Point', 'coordinates': [-122.3998782..."
2,5c6af0068f92e5078765f1c5,games_video,USA,People Powered Radio,2011,37.760524,-122.387799,Fuzz,6,"{'type': 'Point', 'coordinates': [-122.387799,..."
3,5c6af0068f92e5078765f1c9,web,USA,,2011,37.444098,-122.161287,Ziippi,6,"{'type': 'Point', 'coordinates': [-122.1612868..."
4,5c6af0068f92e5078765f1cb,software,USA,,2009,37.784240,-122.274771,BitRock,20,"{'type': 'Point', 'coordinates': [-122.274771,..."
5,5c6af0068f92e5078765f1ce,web,USA,Video-based online test prep,2009,37.871087,-122.270913,Magoosh,13,"{'type': 'Point', 'coordinates': [-122.270913,..."
6,5c6af0068f92e5078765f1d1,web,USA,Every deal. One place.,2010,40.744618,-73.987764,Yipit,23,"{'type': 'Point', 'coordinates': [-73.987764, ..."
7,5c6af0068f92e5078765f1d3,web,USA,used car buying service,2009,37.870457,-122.268354,Carsala,8,"{'type': 'Point', 'coordinates': [-122.268354,..."
8,5c6af0068f92e5078765f1d4,software,USA,"Fast, secure teamwork platform",2011,40.764577,-73.979901,Unison Technologies,30,"{'type': 'Point', 'coordinates': [-73.9799007,..."
9,5c6af0068f92e5078765f1d7,games_video,USA,Film reaching far.,2010,37.805324,-122.405276,Indee,3,"{'type': 'Point', 'coordinates': [-122.4052761..."


#### Exportamos a json

In [100]:
df2=df[['name','category_code', 'country','founded_year','long','lat','number_of_employees']]
df2.to_json('companies_clean.json', orient="records", lines=True)

#### Tenemos dos posibles zonas: en California y Nueva Jersey. Descartamos california ya que a excepción de una empresa de 100 empleados las demás son demasiado pequeñas (3-6 empleados)

#### Ubicamos empresa en Nueva Jersey, en el mismo edificio de las oficinas de Unison Technologies. Alrededor hay empresas de software, games y web de un tamaño similar al nuestro. No hemos encontrado empresas de mayor tamaño.

#### Publicamos Tableau Online con la zona seleccionada

In [ ]:
https://eu-west-1a.online.tableau.com/#/site/iron2019/views/CompanyVideoGames/Hoja1?:iid=1